In [4]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import math
import os
#os.chdir("C:/Users/ACER/anaconda3/data_analys/Lect_04/задача_2_Economics of Cities/")

from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [50]:
pd.set_option('display.max_columns', None)
original_df = pd.read_csv("930941248_T_ONTIME_REPORTING.csv", sep=",", decimal='.')
original_df

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_NM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 33
0,2020,8,24,1,WN,N8680C,1558,DEN,"Denver, CO",Colorado,DSM,"Des Moines, IA",Iowa,1345,1341.0,-4.0,9.0,17.0,1625,1623.0,-2.0,0.0,NaN,0.0,100.0,102.0,76.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,8,24,1,WN,N7811F,1596,DSM,"Des Moines, IA",Iowa,DEN,"Denver, CO",Colorado,1230,1222.0,-8.0,7.0,9.0,1325,1258.0,-27.0,0.0,NaN,0.0,115.0,96.0,80.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,8,24,1,WN,N963WN,450,DSM,"Des Moines, IA",Iowa,LAS,"Las Vegas, NV",Nevada,1545,1620.0,35.0,12.0,5.0,1655,1707.0,12.0,0.0,NaN,0.0,190.0,167.0,150.0,1216.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,8,24,1,WN,N915WN,374,DSM,"Des Moines, IA",Iowa,STL,"St. Louis, MO",Missouri,700,701.0,1.0,12.0,3.0,800,800.0,0.0,0.0,NaN,0.0,60.0,59.0,44.0,259.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,8,24,1,WN,N8680C,1558,DSM,"Des Moines, IA",Iowa,STL,"St. Louis, MO",Missouri,1715,1707.0,-8.0,5.0,4.0,1820,1758.0,-22.0,0.0,NaN,0.0,65.0,51.0,42.0,259.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3367,2020,8,24,1,UA,N831UA,433,DEN,"Denver, CO",Colorado,DSM,"Des Moines, IA",Iowa,1750,1746.0,-4.0,16.0,4.0,2041,2023.0,-18.0,0.0,NaN,0.0,111.0,97.0,77.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN
3368,2020,8,23,7,UA,N822UA,771,DSM,"Des Moines, IA",Iowa,DEN,"Denver, CO",Colorado,753,832.0,39.0,9.0,7.0,841,911.0,30.0,0.0,NaN,0.0,108.0,99.0,83.0,589.0,30.0,0.0,0.0,0.0,0.0,NaN
3369,2020,8,23,7,UA,N827UA,433,DEN,"Denver, CO",Colorado,DSM,"Des Moines, IA",Iowa,1750,1742.0,-8.0,16.0,3.0,2041,2017.0,-24.0,0.0,NaN,0.0,111.0,95.0,76.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN
3370,2020,8,22,6,UA,N851UA,771,DSM,"Des Moines, IA",Iowa,DEN,"Denver, CO",Colorado,753,744.0,-9.0,11.0,8.0,841,824.0,-17.0,0.0,NaN,0.0,108.0,100.0,81.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
df = original_df[[
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'OP_UNIQUE_CARRIER',
 'ORIGIN',
 'ORIGIN_STATE_NM',
 'DEST',
 'DEST_CITY_NAME',
 'DEST_STATE_NM',
 'CRS_DEP_TIME',
 'DEP_DELAY',
 'DISTANCE']]

# Фильтрация по DEP_DELAY
df = df[(df['DEP_DELAY'] != 0) & df['DEP_DELAY'].between(-40, 60)]

# Выбираем только один штат
df = df[df.ORIGIN_STATE_NM == 'Iowa'] # size = 1685
df = df.drop(columns=['DEST_CITY_NAME', 'DEST_STATE_NM'])

# Конвертируем время hhmm в минуты
time = df['CRS_DEP_TIME'].copy()
df['CRS_DEP_TIME'] = time % 100 + 60 * (time // 100)

df.agg({'DEP_DELAY': ['min', 'max', 'mean', 'median']}) #оценка параметров

,DEP_DELAY
min,-37.000000
max,59.000000
mean,-5.222009
median,-7.000000


In [86]:
prep_df = df[[
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'CRS_DEP_TIME',
 'DEP_DELAY',
 'DISTANCE']]
prep_df.head()

,DAY_OF_MONTH,DAY_OF_WEEK,CRS_DEP_TIME,DEP_DELAY,DISTANCE
1,24,1,750,-8.0,589.0
2,24,1,945,35.0,1216.0
3,24,1,420,1.0,259.0
4,24,1,1035,-8.0,259.0
9,25,2,750,-9.0,589.0


In [91]:
carriers = df["OP_UNIQUE_CARRIER"].unique()
print(carriers)
cities = df["ORIGIN"].unique()
print(cities)

prep_df[carriers] = 0

for carrier in carriers:
    prep_df.loc[data_frame['OP_UNIQUE_CARRIER'] == carrier, carrier] = 1

# перемешивание данных
prep_df = prep_df.sample(frac=1) 

['WN' '9E' 'MQ' 'OO' 'UA' 'YV' 'AA' 'YX' 'F9' 'G4' 'OH']
['DSM' 'CID' 'ALO' 'DBQ' 'SUX']


In [92]:
#for col in df.columns: if sum(df[col].isnull())/float(len(df.index)) > 0.8: del df[col] - удалить с процентом nan

X = prep_df.drop(columns=["DEP_DELAY"])
Y = prep_df['DEP_DELAY']

In [93]:
# нормируем колонки по максимуму
X = X / X.aggregate("max")

In [94]:
def prediction(x, b):
    res = 0
    for i in range(0, len(x)):
        res += x[i]*b[i]
    return res+b[-1]

def difference(y, y_):
    return abs(y-y_)

def calculate_error(X,Y, b):
    error=0
    for i in range(0, len(X)):
        x = X.iloc[i].to_list()
        y = Y.iloc[i]
        #error +=  difference(y, prediction(x, b))**2
        pred = 0
        for i in range(0, len(x)):
            pred += x[i]*b[i]
        error += (y-pred-b[-1])**2
    return error

def calculate_error_with_regul(X,Y, b, reg_par):
    error=0
    for i in range(0, len(X)):
        x = X.iloc[i].to_list()
        y = Y.iloc[i]
        #error +=  difference(y, prediction(x, b))**2
        pred = 0
        regul_error = 0
        for i in range(0, len(x)):
            pred += x[i]*b[i]+reg_par*(b[i]**2+abs(b[i]))
            regul_error += b[i]**2+abs(b[i])
        error += (y-pred-b[-1])**2 + reg_par*regul_error
    return error


def gradient(X,Y, b):
    h = 0.00001
    Error = calculate_error(X,Y, b)
    size = len(b)
    grad = [0 for i in range(0, size)]
    for i in range(0, size):
        b[i] += h
        #grad[i] = (Error - calculate_error_by_Kfold(X,Y, b))/h
        grad[i] = (Error - calculate_error(X,Y, b))
        b[i] -= h
    return grad, Error

def gradient_with_regul(X,Y, b,reg_par):
    h = 0.00001
    Error = calculate_error_with_regul(X,Y, b, reg_par)
    size = len(b)
    grad = [0 for i in range(0, size)]
    for i in range(0, size):
        b[i] += h
        grad[i] = (Error - calculate_error_with_regul(X,Y, b, reg_par))
        b[i] -= h
    return grad, Error  

    
def SGD(X,Y, b, reg_par = 0):
    coef_grad = 1
    coef_prev_grad = 0.2
    prev_grad, prev_error =gradient_with_regul(X,Y,b, reg_par)
    for i in range(0, len(b)):
        b[i]+= prev_grad[i] * coef_grad
    c = 0
    error = prev_error+ 1
    next_error = 0
    while (abs(error - prev_error)> 0.001) & (c < 50):
        grad, error = gradient_with_regul(X,Y,b, reg_par)
        c+=1
        
        for i in range(0, len(b)):
            b[i]+= grad[i]*coef_grad + prev_grad[i] * coef_prev_grad
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while_was_not_activated = True
        while next_error > error:
            coef_grad /= 2
            coef_prev_grad /= 2
            for i in range(0, len(b)):
                b[i] -= grad[i]*coef_grad + prev_grad[i] * coef_prev_grad
            next_error = calculate_error_with_regul(X,Y, b, reg_par)
            while_was_not_activated = False
        if while_was_not_activated:
            coef_grad *= 2
            coef_prev_grad *= 2
            
        prev_grad = grad
        prev_error = error
        error = next_error
        print(c, ' :', error)
    return b, calculate_error(X,Y, b)


def Adagrad(X,Y, b, reg_par = 0):
    coef_grad = 0.1
    G_vector = [0.01 for i in range(0, len(b))]
    G_vector_sqr = [i**2 for i in G_vector]
    
    c = 0
    next_error = 0
    while c < 50:
        grad, error = gradient_with_regul(X,Y,b, reg_par)
        c+=1
        for i in range(0, len(b)):
            G_vector_sqr[i] += grad[i]**2
            b[i]+= grad[i]*coef_grad/G_vector[i]
        
        
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while_was_not_activated = True
        while next_error > error:
            coef_grad /= 2
            for i in range(0, len(b)):
                b[i] -= grad[i]*coef_grad/G_vector[i]
            next_error = calculate_error_with_regul(X,Y, b, reg_par)
            while_was_not_activated = False
        if while_was_not_activated:
            coef_grad *= 2
        for i in range(0, len(b)):
            G_vector[i] = G_vector_sqr[i]**0.5
        error = next_error
        print(c, ' :', error, '; coef:', coef_grad)
    return b, calculate_error(X,Y, b)


def RMSprop(X, Y, b, scale_coef=0.5, reg_par = 0):
    coef_grad = 0.1
    G_vector = [0.01 for i in range(0, len(b))]
    G_vector_sqr = [i**2 for i in G_vector]
    
    c = 0
    next_error = 0
    while c < 50:
        grad, error = gradient_with_regul(X, Y, b, reg_par)
        c+=1
        for i in range(0, len(b)):
            G_vector_sqr[i] = G_vector_sqr[i] * scale_coef +  (1-scale_coef) * grad[i]**2
            b[i] += grad[i]*coef_grad/G_vector[i]
        
        
        next_error = calculate_error_with_regul(X, Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while_was_not_activated = True
        while next_error > error:
            coef_grad /= 2
            for i in range(0, len(b)):
                b[i] -= grad[i]*coef_grad/G_vector[i]
            next_error = calculate_error_with_regul(X, Y, b, reg_par)
            while_was_not_activated = False
        if while_was_not_activated:
            coef_grad *= 2
        for i in range(0, len(b)):
            G_vector[i] = G_vector_sqr[i]**0.5
        error = next_error
        print(c, ' :', error, '; coef:', coef_grad)
    return b, calculate_error(X,Y, b)


def Adam(X,Y, b, scale_coef_M=0.5, scale_coef_G=0.5, reg_par = 0):
    # здесь G_vector - то же самое что G_vector_sqr в прошлыых двух
    coef_grad = 0.1
    G_vector = [0.01 for i in range(0, len(b))]
    
    M_vector = [0 for i in range(0, len(b))]
    
    c = 0
    next_error = 0
    while c < 50:
        grad, error = gradient_with_regul(X,Y,b, reg_par)
        c+=1
        for i in range(0, len(b)):
            M_vector[i] = M_vector[i] * scale_coef_M + (1-scale_coef_M)* grad[i]
            G_vector[i] = G_vector[i] * scale_coef_G + (1-scale_coef_G)* grad[i]**2
            
            m_ = M_vector[i]/(1-scale_coef_M**c)
            g_ = (G_vector[i]/ (1-scale_coef_G**c))**0.5
            
            b[i]+= m_*coef_grad/g_
        
        
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while_was_not_activated = True
        while next_error > error:
            coef_grad /= 2
            for i in range(0, len(b)):
                m_ = M_vector[i]/(1-scale_coef_M**c)
                g_ = (G_vector[i]/ (1-scale_coef_G**c))**0.5
                b[i]-= m_*coef_grad/g_
                
            next_error = calculate_error_with_regul(X,Y, b, reg_par)
            while_was_not_activated = False
        if while_was_not_activated:
            coef_grad *= 2
        print(c, ' :', error, '; coef:', coef_grad)
    return b, calculate_error(X,Y, b)


def K_fold_by_reg_param(X, Y, b, reg_params):
    size_test = len(X)//len(reg_params)
    begin = 0
    end = size_test + len(X)%len(reg_params)
    best_b=[]
    best_error = 1000000000# не знаю как поставить бесконечность
    best_reg_param = reg_params[0]
    for i in range(0, len(reg_params)):
        
        X_train = pd.concat([X.iloc[0:begin],X.iloc[end:len(X)]])
        Y_train = pd.concat([Y.iloc[0:begin],Y.iloc[end:len(Y)]])
        b, error = SGD(X_train, Y_train, b, reg_params[i])
        
        error = calculate_error(X.iloc[begin:end],Y.iloc[begin:end], b)
        if error < best_error:
            best_error = error
            best_b = b
            best_reg_param = reg_params[i]
            
        begin += size_test
        end += size_test
        
    return best_b, best_error, best_reg_param

In [96]:
b = [1 for _ in range(len(columns))]
g = Y.mean()

In [99]:
'''b = [-0.030386120488803316,
 -0.029674049015335522,
 -0.0035100154031383105,
 -0.003200133522719021,
 -0.001445508196131786,
 0.0012666197926167317,
 -0.014518818537591143,
 -0.013747216772204616,
 -0.004716126209607403,
 -0.016204393326723784,
 -0.010267026861892499,
 0.004420402418926793,
 0.0033827350743404943,
 -0.01311095885014846,
 0.009417268801753656, g]'''
RMSprop(X, Y, b, 0.8, 1)
# 3634581 - maen

1  : 289638.76162113243 ; coef: 0.003125
2  : 289265.54126396286 ; coef: 0.00625
3  : 288440.8434684364 ; coef: 0.0125
4  : 286634.4423325681 ; coef: 0.025
5  : 282748.13131398626 ; coef: 0.05
6  : 274694.09330330585 ; coef: 0.1
7  : 259249.34292159305 ; coef: 0.2
8  : 233896.84550502346 ; coef: 0.4
9  : 219818.9571402699 ; coef: 0.8
10  : 211135.84883953116 ; coef: 0.2
11  : 208945.97905110367 ; coef: 0.1
12  : 206795.68893404806 ; coef: 0.2
13  : 205305.3842235967 ; coef: 0.05
14  : 204424.561603971 ; coef: 0.025
15  : 204217.8143648534 ; coef: 0.05
16  : 203975.10195955384 ; coef: 0.025
17  : 203246.7311820064 ; coef: 0.0125
18  : 202950.91376319109 ; coef: 0.00625
19  : 202892.5929725039 ; coef: 0.0125
20  : 202834.20899602692 ; coef: 0.00625
21  : 202803.59497714235 ; coef: 0.0125
22  : 202591.572793126 ; coef: 0.00625
23  : 202319.17422803564 ; coef: 0.003125
24  : 202247.2983336754 ; coef: 0.0015625
25  : 202220.26335489238 ; coef: 0.003125
26  : 202182.62303113734 ; coef: 0.001

([-0.010519399615439145,
  -0.058789708811864835,
  -0.01093336858964271,
  -0.0002328883710707248,
  -0.000150119966545899,
  -7.838996678616745e-05,
  6.999314072704532e-05,
  -2.7422420382059657e-05,
  0.00012368468172519148,
  -3.1430580875165394e-05,
  7.621060355449892e-05,
  2.305466674628492e-06,
  -0.00011191108755833728,
  6.752573307983214e-05,
  -0.26599153375821094],
 194284.2884885188)

In [24]:
b = [-3.8600182477692075,
 -8.669282576286276,
 -2.931755242622002,
 5.310154304652749,
 5.251652500629584,
 2.642202468466634,
 0.18067101427803842,
 -1.1279689148534424,
 -3.4674051775662424,
 -3.717961226445898,
 4.375254739158854,
 3.4196021354113357,
 0.7770875378445566,
 1.216595131291389,
 -5.523187531051819,
 -6.057591062421216,
 0.3066753770708594,
 -4.0049240225419815,
 1.9006025497972634,
 8.783777045830218]
b = [0 for i in range(0, len(columns))]
Adam(X,Y,b,g, 0.1, 0.8)
# 3634581 - maen

1  : 116105.842163796 ; coef: 0.00625
2  : 83296.72467108814 ; coef: 0.0125
3  : 61860.24249923296 ; coef: 0.00625
4  : 59329.600737332126 ; coef: 0.003125
5  : 52082.18586786673 ; coef: 0.00625
6  : 51910.409722963195 ; coef: 0.003125
7  : 51866.93403072774 ; coef: 0.00625
8  : 51805.17358430002 ; coef: 0.003125
9  : 51756.46146285248 ; coef: 0.00625
10  : 51713.36834941317 ; coef: 0.0015625
11  : 51650.39771908612 ; coef: 0.003125
12  : 51606.283669338736 ; coef: 0.00625
13  : 51532.49585198361 ; coef: 0.003125
14  : 51479.21626501691 ; coef: 0.0015625
15  : 51474.01996962203 ; coef: 0.00078125
16  : 51383.69963174434 ; coef: 0.0015625
17  : 51355.16111486852 ; coef: 0.003125
18  : 51304.04761827563 ; coef: 0.00625
19  : 51204.544307882934 ; coef: 0.00078125
20  : 51182.29790735339 ; coef: 0.0015625
21  : 51155.697843667804 ; coef: 0.00078125
22  : 51148.57190076099 ; coef: 0.000390625
23  : 51102.397072901855 ; coef: 0.00078125
24  : 51083.750691837864 ; coef: 0.0015625
25  : 51052.

[-0.030386120488803316,
 -0.029674049015335522,
 -0.0035100154031383105,
 -0.003200133522719021,
 -0.001445508196131786,
 0.0012666197926167317,
 -0.014518818537591143,
 -0.013747216772204616,
 -0.004716126209607403,
 -0.016204393326723784,
 -0.010267026861892499,
 0.004420402418926793,
 0.0033827350743404943,
 -0.01311095885014846,
 0.009417268801753656]

In [78]:
b = [0.07108979547236088,
 -0.9733133421334863,
 0.9968873042940036,
 4.109941361898274,
 2.059924274732903,
 0.7515062699570404,
 -2.3957530262211173,
 -1.5086747078487333,
 -4.630649158547469,
 -1.84807660184145,
 -1.8403365958128066,
 -2.8559587160828506,
 2.721951107553011,
 -9.714869260404695,
 -2.1713094116244145,
 -4.869790876994323]
error=0
for i in range(0, len(X)):
    x = X.iloc[i].to_list()
    y = Y.iloc[i]
    #error +=  difference(y, prediction(x, b, g))**2
    pred = 0
    for i in range(0, len(x)):
        pred += x[i]*b[i]
    error += abs(y-pred-b[-1])
(error/len(X))

5.28897812869417

In [79]:
error=0
for i in range(0, len(X)):
    y = Y.iloc[i]
    error += abs(y-g)
error / len(X)

9.287431294945453

In [55]:
prep_df.agg({'CRS_DEP_TIME': 'min' })[0]

0.5120910384068279

In [ ]:
last_b = [-0.030386120488803316,
 -0.029674049015335522,
 -0.0035100154031383105,
 -0.003200133522719021,
 -0.001445508196131786,
 0.0012666197926167317,
 -0.014518818537591143,
 -0.013747216772204616,
 -0.004716126209607403,
 -0.016204393326723784,
 -0.010267026861892499,
 0.004420402418926793,
 0.0033827350743404943,
 -0.01311095885014846,
 0.009417268801753656] # на -20...20

b = [-1.0916733579732403,
 -2.5640838101518204,
 2.24665028274803,
 3.4670553823936916,
 2.311274230184649,
 1.2462326201785583,
 -2.147353830572917,
 -1.0697943903179663,
 -2.2125041864212505,
 -2.023801340550869,
 -1.373493009975614,
 -2.3276501602169244,
 2.156534080207264,
 -7.107128976965588,
 -2.2446902880543216,
 -3.4282117381240815] # -40...90

In [82]:
Y.agg({'DEP_DELAY': 'mean' })[0]
pd.concat([data_frame.iloc[0:10],data_frame.iloc[20:30]]).head(20)

-5.222008957133717